In [ ]:
from repenseai.genai.agent import Agent
from repenseai.genai.tasks.api import Task

from repenseai.genai.tasks.conditional import ConditionalTask
from repenseai.genai.tasks.workflow import Workflow

from pydantic import BaseModel

class Intention(BaseModel):
    intention: int


# Agents

intention_agent = Agent(
    model="gpt-4o", 
    model_type="chat", 
    temperature=0.0,
    json_schema=Intention
)

chat_agent = Agent(
    model="gpt-4o", 
    model_type="chat", 
    temperature=0.0
)

search_agent = Agent(
    model="sonar-pro",
    model_type="search",
    temperature=0.0
)

reasoning_agent = Agent(
    model="deepseek-reasoner",
    model_type="chat",
    temperature=0.0
)

image_agent = Agent(
    model="black-forest-labs/FLUX.1.1-pro",
    model_type="image",
    temperature=0.0
)

# Tasks

intention_task = Task(
    user=(
        "You are a chatbot in a messaging app.\n"
        "Choose an option base on user intention:\n"
        "1 Search for information\n"
        "2 Chat with the user\n"
        "3 Reason with the user (math problems or logic questions)\n"
        "4 Generate an image\n\n"
        "Your output should have only the intention number as a integer:\n\n{message}"
    ), 
    agent=intention_agent,
    simple_response=True
)

shared_prompt = "You are a chatbot in a messaging app. Respond to the user's message: {message}"

chat_task = Task(
    user=shared_prompt,
    agent=chat_agent,
    simple_response=True
)

search_task = Task(
    user=shared_prompt,
    agent=search_agent,
    simple_response=True
)

reasoning_task = Task(
    user=shared_prompt,
    agent=reasoning_agent,
    simple_response=True
)

image_task = Task(
    user="{message}",
    agent=image_agent,
    simple_response=True
)

# Router

intention_split = ConditionalTask(
    condition=lambda context: int(context.get("intention", {}).get("intention")),
    tasks={
        1: search_task,
        2: chat_task,
        3: reasoning_task,
        4: image_task,
    },
    default_task=chat_task,
)

# Chatbot Workflow

workflow = Workflow(
    [
        [intention_task, "intention"],
        [intention_split, "output"],
    ]
)

# Run the workflow

# response = workflow.run({"message": "qual a derivada de 2x**2?"})
# response = workflow.run({"message": "gere a imagem de um peixe"})
# response = workflow.run({"message": "quanto foi o ultimo jogo do palmeiras?"})
response = workflow.run({"message": "opa! tudo bem?"})

intention = response.get("intention", {}).get("intention")

print()
print(f"Intention: {intention}")

if intention == 4:
    from repenseai.utils.image import display_base64_image
    display_base64_image(response['output'])
else:
    print(f"Resposta: {response['output']}")